In [1]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from extract_line_feat_v3 import *

In [2]:
img = read_image("../Data_Chula/1 Sep HER2 2+ DISH - case 1/1 Sep D- HER2 2+ case 1 (27).JPG")
cont_mask, h_cont_mask = extract_mask(img)

In [24]:
from skimage.feature import local_binary_pattern
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy

holes = []
min_cont = 0.1
min_area_hole = 25
q_cont_mask = np.zeros(cont_mask.shape).astype(np.uint8)
d = .005
ths = np.arange(min_cont, 1., d)
for i in range(len(ths[:-1])):
    th1 = ths[i]
    th2 = ths[i+1]
    holes.append(count_hole(cont_mask > th1, min_area_hole))
    q_cont_mask += (
        (cont_mask >= th1) & 
        (cont_mask < th2)
    ).astype(np.uint8) * i


lbp = local_binary_pattern(q_cont_mask, P=8, R=1, method="uniform")
lbp = lbp[cont_mask >= 0.1]
hists = np.histogram(lbp, bins=10)[0]
hists = hists / hists.sum()

glcm = graycomatrix(
    q_cont_mask, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], 
    levels=len(ths), symmetric=True, normed=False
).astype(float)

for i in range(4):
    glcm[0,:,0,i] = 0
    glcm[:,:,0,i] /= glcm[:,:,0,i].sum()

contrast = graycoprops(glcm, "contrast").reshape(-1).mean()
dissim = graycoprops(glcm, "dissimilarity").reshape(-1).mean()
homo = graycoprops(glcm, "homogeneity").reshape(-1).mean()
asm = graycoprops(glcm, "ASM").reshape(-1).mean()
energy = graycoprops(glcm, "energy").reshape(-1).mean()
corrs = graycoprops(glcm, "correlation").reshape(-1).mean()
entropy = np.array([shannon_entropy(glcm[:,:,0,i]) for i in range(4)]).mean()

print(contrast, dissim, homo, asm, energy, corrs, entropy)

575.4665233088183 16.84413972640164 0.08080601215786537 0.0017691125642612575 0.04199792025876574 0.6234151086751989 1.9477995080138486


In [63]:
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy

glcm = graycomatrix(
    q_cont_mask, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], 
    levels=len(th_ranges), symmetric=True, normed=False
).astype(float)

for i in range(4):
    glcm[0,:,0,i] = 0
    glcm[:,:,0,i] /= glcm[:,:,0,i].sum()

contrast = graycoprops(glcm, "contrast").reshape(-1).mean()
dissim = graycoprops(glcm, "dissimilarity").reshape(-1).mean()
homo = graycoprops(glcm, "homogeneity").reshape(-1).mean()
asm = graycoprops(glcm, "ASM").reshape(-1).mean()
energy = graycoprops(glcm, "energy").reshape(-1).mean()
corrs = graycoprops(glcm, "correlation").reshape(-1).mean()
entropy = np.array([shannon_entropy(glcm[:,:,0,i]) for i in range(4)]).mean()

In [65]:
entropy

np.float64(0.0025050450785144263)